In [132]:
import numpy as np
import pandas as pd
import scipy.stats as stat
from scipy.interpolate import interp1d
CONF_LVL = 5

In [133]:
sofr_curve = pd.read_csv("data/sofr_curve.csv")
sofr_curve_tenors = sofr_curve[['T','Tenor']]
sofr_curve = sofr_curve.drop(['T','Tenor','Unnamed: 253'], axis=1).T
sofr_curve.columns = sofr_curve_tenors['T']
sofr_curve.index = pd.to_datetime(sofr_curve.index, dayfirst=True).date
sofr_curve = sofr_curve.sort_index()
sofr_curve

/var/folders/yl/d1lf3psd5vj7gcckrfs57qyh0000gn/T/ipykernel_21274/2426952476.py:5: UserWarning: Parsing dates in %Y%m%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  sofr_curve.index = pd.to_datetime(sofr_curve.index, dayfirst=True).date


T,0.002778,0.083333,0.166667,0.250000,0.500000,0.750000,1.000000,2.000000,3.000000,4.000000,...,15.000000,16.000000,17.000000,18.000000,19.000000,20.000000,25.000000,30.000000,35.000000,40.000000
2022-10-31,0.039191,0.038721,0.038670,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.032350,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.046970,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.036010,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.039100,0.040852,0.044884,0.046580,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.039350,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.047500,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.037380,0.037152,0.036870,0.036534,0.034424,0.032558,0.030723,0.028933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.053105,0.053084,0.053202,0.053424,0.053690,0.053293,0.052503,0.048399,0.045999,0.044850,...,0.044093,0.044056,0.043972,0.043841,0.043663,0.043437,0.041874,0.040238,0.038438,0.036493
2023-10-25,0.052981,0.053057,0.053259,0.053490,0.053749,0.053393,0.052653,0.048791,0.046595,0.045594,...,0.045289,0.045268,0.045198,0.045078,0.044906,0.044683,0.043076,0.041358,0.039564,0.037641
2023-10-26,0.053047,0.053089,0.053218,0.053414,0.053567,0.053089,0.052243,0.048044,0.045645,0.044538,...,0.044269,0.044244,0.044171,0.044049,0.043882,0.043668,0.042182,0.040589,0.038760,0.036764
2023-10-27,0.052989,0.053040,0.053185,0.053368,0.053486,0.052991,0.052115,0.047758,0.045284,0.044200,...,0.044460,0.044461,0.044411,0.044309,0.044157,0.043955,0.042508,0.040985,0.039167,0.037148


1. Full revaluation formula of the payer swap:
- Forward Rate:
$$
    F_{i}(t) = \dfrac{D_{i}(t)-D_{i+1}(t)}{\Delta_{i} \cdot D_{i+1}(t)}
$$
- Floating Leg PV:
\begin{equation*}
  \begin{split}
    PV_{flt} &= \sum^{10}_{i=1} \Delta_{i-1} \cdot F_{i-1}(t) \cdot D_i(t) \\
    &= 1 - D_{10}(t)
  \end{split}
\end{equation*}

- Fixed Leg PV:
\begin{equation*}
  \begin{split}
    PV_{fix} &= R_{swap} \sum^{10}_{i=1} \Delta_{i-1} \cdot D_i(t) \\
    &= R_{swap} \cdot PV01
  \end{split}
\end{equation*}

- Payer swap PV:
\begin{equation*}
  \begin{split}
    PV_{payer} &= Notional \cdot (PV_{flt} - PV_{fix}) \\
    & = Notional \cdot [1 - D_{10}(t) - R_{swap} \cdot PV01]
  \end{split}
\end{equation*}



In [134]:
def get_discount_curve(
        zero_rates:list,
        tenors:list # day count fraction from today to each payment day
        ):
    assert len(zero_rates) == len(tenors), f"Expect {len(tenors)} zero rates, got {len(zero_rates)}."
    Z = np.array(zero_rates)
    T = np.array(tenors)
    return np.exp(-Z*T)


def get_forward_curve(
        zero_rates:list,
        tenors:list, 
        interval=1, # day count fraction of the interval between payment days
        ):
    DF = get_discount_curve(zero_rates, tenors)
    DF_start = np.concatenate([[1], DF[:-1]]) # first DF=1. we dont consider forward swap here
    DF_end = DF
    F = (DF_start - DF_end) / (DF_end * interval)
    return F
    

def get_payer_swap_pv(
        zero_rates:list,
        tenors:list,
        forward_rates:list=None,
        swap_rate=0.042,
        interval=1, 
        notional=100*1000000
        ):
    DF = get_discount_curve(zero_rates, tenors)
    F = get_forward_curve(zero_rates, tenors) if forward_rates is None else np.array(forward_rates)
    pv_fix = swap_rate*sum(interval*DF)
    pv_flt = sum(interval*F*DF)
    return notional*(pv_flt - pv_fix)


def swap_pnl_1d_full(
        zero_rates_t0:list,
        zero_rates_t1:list,
        tenors:list
    ):
    pv_t0 = get_payer_swap_pv(zero_rates_t0, tenors)
    pv_t1 = get_payer_swap_pv(zero_rates_t1, tenors)
    return pv_t1 - pv_t0

def swap_pnl_1d_sens(
        zero_rates_t0:list,
        zero_rates_chng:list,
        tenors:list,
        swap_rate=0.042,
        notional=100*1000000
        ):    
    zero_rates_chng = np.array(zero_rates_chng)
    tenors = np.array(tenors)
    DF_t0 = get_discount_curve(zero_rates_t0, tenors)
    W = notional * swap_rate * tenors * DF_t0  # weight of risk factors in PnL sensitivity
    W[-1] = notional * (1+swap_rate) * tenors[-1] * DF_t0[-1]
    return W @ zero_rates_chng

In [143]:
# we only using 1 to 10
swap_tenors_10y = range(1,10+1)
# filter sor_curve
ten_year_sofr_curve = sofr_curve[swap_tenors_10y]

sofr_curve_t0 = ten_year_sofr_curve.iloc[0]
sofr_curve_exclude_first = ten_year_sofr_curve.values[1:]

swap_pnl_full_hist = [] 
swap_pnl_sensitivity = []

for current_sofr_curve in sofr_curve_exclude_first:
    pnl_full_evaulation = swap_pnl_1d_full(
        zero_rates_t0=sofr_curve_t0,
        zero_rates_t1=current_sofr_curve,
        tenors=swap_tenors_10y
    )
    zero_rates_chng = np.array(sofr_curve_t0)-np.array(current_sofr_curve)
    pnl_swap_evaluation = swap_pnl_1d_sens(
        zero_rates_t0=sofr_curve_t0,
        zero_rates_chng=zero_rates_chng,
        tenors=swap_tenors_10y
    )
    swap_pnl_full_hist.append(pnl_full_evaulation)
    swap_pnl_sensitivity.append(pnl_swap_evaluation)
np.shape(swap_pnl_full_hist)

(250,)

# Stocks

In [144]:
S_appl = pd.read_csv("data/AAPL.csv").set_index('Date')
S_boa = pd.read_csv("data/BAC.csv").set_index('Date')
S_ford = pd.read_csv("data/F.csv").set_index('Date')
S_msft = pd.read_csv("data/MSFT.csv").set_index('Date')

S_appl.index = pd.to_datetime(S_appl.index, dayfirst=True).date
S_boa.index = pd.to_datetime(S_boa.index, dayfirst=True).date
S_ford.index = pd.to_datetime(S_ford.index, dayfirst=True).date
S_msft.index = pd.to_datetime(S_msft.index, dayfirst=True).date

S_appl = S_appl.sort_index()
S_boa = S_boa.sort_index()
S_ford = S_ford.sort_index()
S_msft = S_msft.sort_index()

/var/folders/yl/d1lf3psd5vj7gcckrfs57qyh0000gn/T/ipykernel_21274/3450744268.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  S_appl.index = pd.to_datetime(S_appl.index, dayfirst=True).date
/var/folders/yl/d1lf3psd5vj7gcckrfs57qyh0000gn/T/ipykernel_21274/3450744268.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  S_boa.index = pd.to_datetime(S_boa.index, dayfirst=True).date
/var/folders/yl/d1lf3psd5vj7gcckrfs57qyh0000gn/T/ipykernel_21274/3450744268.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  S_ford.index = pd.to_datetime(S_ford.index, dayfirst=True).date
/var/f

In [145]:
# combine 4 stock prices into a return matrix, w same date range
offset = \
    min(
        S_appl.index.min(),
        S_boa.index.min(),
        S_ford.index.min(),
        S_msft.index.min()
    )

idx_appl = ((S_appl.index - offset) / pd.offsets.Day(1)).to_list()
idx_boa = ((S_boa.index - offset) / pd.offsets.Day(1)).to_list()
idx_ford = ((S_ford.index - offset) / pd.offsets.Day(1)).to_list()
idx_msft = ((S_msft.index - offset) / pd.offsets.Day(1)).to_list()

interp_appl =  interp1d(idx_appl, S_appl['Adj Close'].to_list())
interp_boa =  interp1d(idx_boa, S_boa['Adj Close'].to_list())
interp_ford =  interp1d(idx_ford, S_ford['Adj Close'].to_list())
interp_msft =  interp1d(idx_msft, S_msft['Adj Close'].to_list())

In [146]:
idx_all = list(set(idx_appl + idx_boa + idx_ford + idx_msft)) # to merge the index of all 4 stocks
idx_all = sorted(idx_all)
# relative daily change of the 4 stocks
R_stocks = \
    pd.DataFrame(
        {
            'appl': pd.Series(interp_appl(idx_all)).pct_change().dropna(),
            'boa': pd.Series(interp_boa(idx_all)).pct_change().dropna(),
            'ford': pd.Series(interp_ford(idx_all)).pct_change().dropna(),
            'msft': pd.Series(interp_msft(idx_all)).pct_change().dropna()
        }
        )
R_stocks.head(5)

,appl,boa,ford,msft
1,-0.017543,0.004439,0.002244,-0.017059
2,-0.037305,-0.003039,-0.025373,-0.035368
3,-0.042405,-0.005542,0.015314,-0.026579
4,-0.001947,0.025077,0.018854,0.033326
5,0.003902,0.005980,0.014064,0.029270


In [147]:
def stocks_pnl_1d_full(R, 
                       W=[1E6, 1E6, 1E6, 1E6]
                       ):
    assert len(R) == 4, f"Expect 4 stock returns, got {len(R)}."
    return (W[0]*((1+R[0])-1) + W[1]*((1+R[1])-1) + W[2]*((1+R[2])-1) + W[3]*((1+R[3])-1))

def stocks_pnl_1d_sens(R, 
                       W=[1E6, 1E6, 1E6, 1E6]
                       ):
    assert len(R) == 4, f"Expect 4 stock returns, got {len(R)}."
    return R@W

In [148]:
pnl_1d_full_stocks_hist = [stocks_pnl_1d_full(r) for r in R_stocks.values]
pn1_1d_sen_stocks_hist = [stocks_pnl_1d_sens(r) for r in R_stocks.values]

In [152]:
pnl_1d_full_hist = np.array(pnl_1d_full_stocks_hist) + np.array(swap_pnl_full_hist)

var_1d_full_hist = np.abs(np.percentile(pnl_1d_full_hist, CONF_LVL))

pnl_1d_sens = np.array(pn1_1d_sen_stocks_hist) + np.array(swap_pnl_sensitivity)
var_1d_sensi = np.abs(np.percentile(pnl_1d_sens, CONF_LVL))


In [151]:

print("============================================================================================================================")
print("Historical VaR:")
print(f"VaR [1d, {CONF_LVL}%], Full Revaluation: {var_1d_full_hist:,.0f}") 
print(f"VaR [1d, {CONF_LVL}%], Sensitivity Revaluation: {var_1d_sensi:,.0f}") 

# print(f"VaR [1d, {CONF_LVL}%], Sensitivity: {var_1d_sens_mc:,.0f}") 
print("============================================================================================================================")

Historical VaR:
VaR [1d, 5%], Full Revaluation: 6,202,470
VaR [1d, 5%], Sensitivity Revaluation: 4,793,273
